In [45]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from matplotlib import pyplot as plt
from time import time
from typing import List
from sklearn.model_selection import train_test_split

In [47]:
data = pd.read_csv('data/train.csv')
submission_test_data = pd.read_csv('data/test.csv')

In [48]:
display(data)
display(submission_test_data)

,case_id,equipment_id,completion_date,action_recommendation_id,action_recommendation_type,action_recommendation_category,equipment_area,usage_type,speed_category,load_category,floors_category,equipment_category,feedback
0,b7c775ad-4ebe-4848-9c53-37e7c5658e21,ele0000754,2018-10-03,ar00000174,art01,arc03,ga00000803,ut012,7,6,8,tp006,1
1,b177eefd-3946-4949-9699-0a91879350f9,ele0000789,2018-10-03,ar00000248,art02,arc03,ga00000269,ut005,4,6,2,tp001,1
2,fc2d568a-c53c-43ef-8871-a49ec22ab3b1,ele0001227,2018-10-03,ar00000158,art02,arc03,ga00000064,ut011,6,6,7,tp006,1
3,8e645922-1268-4c6b-ae6b-7b1605689cca,ele0001754,2018-10-03,ar00000105,art01,arc03,ga00000662,ut005,8,7,8,tp014,1
4,b44f10b1-9238-44db-8f0c-2d68e8c015a7,ele0002087,2018-10-03,ar00000148,art02,arc03,ga00001256,ut011,7,6,8,tp013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115470,8eda4489-c34c-4e8a-b592-ad57b5fd842a,ele0029759,2020-01-30,ar00000124,art01,arc03,ga00000817,ut011,6,7,8,tp005,1
115471,0113eba6-6928-461a-b994-35a0b2eb9f4e,ele0029783,2020-01-30,ar00000291,art02,arc01,ga00001027,ut001,4,3,1,tp015,1
115472,c2eab0dc-218a-4a95-ab28-47449f42f660,ele0029785,2020-01-30,ar00000048,art01,arc02,ga00000355,ut011,2,1,4,tp002,1
115473,c2eab0dc-218a-4a95-ab28-47449f42f660,ele0029785,2020-01-30,ar00000293,art01,arc02,ga00000355,ut011,2,1,4,tp002,1


,case_id,equipment_id,completion_date,action_recommendation_id,action_recommendation_type,action_recommendation_category,equipment_area,usage_type,speed_category,load_category,floors_category,equipment_category
0,9d54504e-c805-4859-b92e-a8df797323e7,ele0030053,2019-06-04,ar00000250,art01,arc03,ga00000073,ut005,7,6,8,tp013
1,554e89db-0d65-44f1-a3cb-d79662cd1058,ele0030057,2019-06-04,ar00000188,art01,arc03,ga00000061,ut001,4,7,1,tp005
2,6cb91017-5e91-446b-9064-409758334cea,ele0030180,2019-06-04,ar00000005,art01,arc02,ga00000600,ut011,4,3,4,tp003
3,496f3fe9-36a6-4b44-a686-5383584c4653,ele0030224,2019-06-04,ar00000124,art01,arc03,ga00000136,ut011,5,4,6,tp008
4,f8189755-b79e-4114-8a2b-84e08a2263f9,ele0030580,2019-06-04,ar00000250,art01,arc03,ga00000529,ut011,6,4,7,tp006
...,...,...,...,...,...,...,...,...,...,...,...,...
29423,62bf183c-eb55-4da1-b654-f245b320c6c0,ele0039768,2020-01-30,ar00000082,art03,arc03,ga00000178,ut008,4,8,2,tp005
29424,62bf183c-eb55-4da1-b654-f245b320c6c0,ele0039920,2020-01-30,ar00000024,art03,arc02,ga00000752,ut008,1,5,3,tp002
29425,e1cad254-5772-4f88-836c-03aa75f37341,ele0039921,2020-01-30,ar00000209,art01,arc02,ga00000994,ut011,3,2,5,tp002
29426,29ad636c-522c-46c0-abb8-e29cca030fea,ele0039994,2020-01-30,ar00000174,art01,arc03,ga00001126,ut011,6,4,7,tp012


In [49]:
data[data['feedback'] == 0]
n_ids = data['action_recommendation_id'].nunique()
n_types = data['action_recommendation_type'].nunique()
n_categories = data['action_recommendation_category'].nunique()

print(n_ids, n_types, n_categories)

295 3 6


In [50]:
global_mean_train = data['feedback'].mean()
print(f'Mean of feedback in train data: {global_mean_train:.2f}')

data['equipment_area'].nunique()
# plt.hist(train_data['equipment_area'])
# plt.show()
for column in data.columns:
    print(f'{column}: {data[column].nunique()}')
#     train_data[column].hist()

Mean of feedback in train data: 0.88
case_id: 73945
equipment_id: 30000
completion_date: 466
action_recommendation_id: 295
action_recommendation_type: 3
action_recommendation_category: 6
equipment_area: 1285
usage_type: 13
speed_category: 8
load_category: 8
floors_category: 8
equipment_category: 20
feedback: 2


In [52]:

print('\n\nIDs IN CATEGORIES')
for category in data['action_recommendation_category'].unique():
    print(f'\n{category}')
    cat_ids = data.loc[data['action_recommendation_category'] == category, 'action_recommendation_id'].unique()
    not_cat_ids = data.loc[data['action_recommendation_category'] != category, 'action_recommendation_id'].unique()
    print(np.intersect1d(cat_ids, not_cat_ids))
  
print('\n\nIDs IN TYPES')
for type in data['action_recommendation_type'].unique():
    print(f'\n{type}')
    type_ids = data.loc[data['action_recommendation_type'] == type, 'action_recommendation_id'].unique()
    not_type_ids = data.loc[data['action_recommendation_type'] != type, 'action_recommendation_id'].unique()
    print(np.intersect1d(type_ids, not_type_ids))
 
print('\n\nCATEGORIES IN TYPES')
for type in data['action_recommendation_type'].unique():
    print(f'\n{type}')
    type_cats = data.loc[data['action_recommendation_type'] == type, 'action_recommendation_category'].unique()
    not_type_cats = data.loc[data['action_recommendation_type'] != type, 'action_recommendation_category'].unique()
    print(np.intersect1d(type_cats, not_type_cats))



IDs IN CATEGORIES

arc03
[]

arc01
[]

arc02
[]

arc05
[]

arc04
[]

arc06
[]


IDs IN TYPES

art01
[]

art02
[]

art03
[]


CATEGORIES IN TYPES

art01
['arc01' 'arc02' 'arc03' 'arc04' 'arc06']

art02
['arc01' 'arc02' 'arc03' 'arc04' 'arc05' 'arc06']

art03
['arc01' 'arc02' 'arc03' 'arc05' 'arc06']


In [61]:
def target_encode_mean(df: pd.DataFrame, column: str, min_category_size: int, target_column: str) -> (pd.Series, pd.Series):
    """
    Converts a categorical variable to a target encoded variable based on the means of the target variable.
    """
    # NaN cannot be used as a key so we have to convert NaNs to string
    feature_orig = df[column].replace(np.nan, 'NaN')
    counts_distinct = feature_orig.value_counts()
    counts = feature_orig.map(counts_distinct)
    mean_mapping = df.groupby(column).mean()[target_column]
    feature_mapped = feature_orig.map(mean_mapping)
    feature_mapped[counts < min_category_size] = df[target_column].mean()
    return feature_mapped, mean_mapping


def build_features(df: pd.DataFrame, categorical_features: list, target_column: str):
    print('Starting feature engineering.')
    start_time = time()
    # Category must include at least this many samples. If not, then it's encoded to global mean of the target variable.
    min_category_size = 100
    mean_mappings = {}
    #     In our specific dataset, all of the columns are categorical
    features = categorical_features
    X = pd.DataFrame(columns=features)
    for feature in categorical_features:
        X[feature], mean_mappings[feature] = target_encode_mean(df, feature, min_category_size, target_column)
        
    y = df[target_column]

    print(f'Features and target split. Took {time()-start_time:.2f} s.')
    return X, y, mean_mappings

def map_features(df: pd.DataFrame, feature_columns: List[str], mappings: List[pd.Series], target_column: str):
    X = pd.DataFrame(columns=mappings.keys())
    for column in mappings.keys():
        X[column] = df[column].map(mappings[column])
    y = df[target_column]
    return X, y

categorical_features_binary = [
    'action_recommendation_id',
    'action_recommendation_type',
    'action_recommendation_category',
    'equipment_area',
    'usage_type',
    'speed_category',
    'load_category',
    'floors_category',
    'equipment_category',
]

train_data, test_data = train_test_split(data, test_size=0.2)
X_train, y_train, feature_mappings_binary = build_features(train_data, categorical_features_binary, 'feedback')
X_test, y_test = map_features(test_data, categorical_features_binary, feature_mappings_binary, 'feedback')

display(X_test)
display(feature_mappings_binary)

Starting feature engineering.
Features and target split. Took 0.30 s.


,action_recommendation_id,action_recommendation_type,action_recommendation_category,equipment_area,usage_type,speed_category,load_category,floors_category,equipment_category
104947,0.871235,0.849059,0.910544,0.986301,0.880680,0.879470,0.910063,0.873349,0.922481
79458,0.897753,0.849059,0.910544,0.967213,0.911693,0.909253,0.880642,0.872089,0.926073
33728,0.952288,0.849059,0.910544,0.972527,0.880680,0.893960,0.910063,0.886434,0.924892
15669,0.871235,0.849059,0.910544,0.989583,0.880680,0.879470,0.859220,0.872089,0.886251
83681,0.710602,0.921033,0.904404,0.927273,0.880680,0.879470,0.910063,0.861879,0.906380
...,...,...,...,...,...,...,...,...,...
77471,0.943802,0.929870,0.910544,0.850000,0.880680,0.909253,0.910063,0.886434,0.886251
22278,0.952288,0.849059,0.910544,0.936652,0.861953,0.893960,0.910063,0.912194,0.863526
90986,0.775000,0.849059,0.910544,0.969697,0.880680,0.744936,0.789905,0.861879,0.784738
33280,0.913212,0.929870,0.910544,1.000000,0.916508,0.902197,0.880642,0.912194,0.914653


{'action_recommendation_id': action_recommendation_id
 ar00000001    0.947536
 ar00000002    1.000000
 ar00000003    1.000000
 ar00000004    1.000000
 ar00000005    0.813953
                 ...   
 ar00000291    0.935780
 ar00000292    0.993631
 ar00000293    0.821738
 ar00000294    0.738636
 ar00000295    0.452830
 Name: feedback, Length: 287, dtype: float64,
 'action_recommendation_type': action_recommendation_type
 art01    0.849059
 art02    0.929870
 art03    0.921033
 Name: feedback, dtype: float64,
 'action_recommendation_category': action_recommendation_category
 arc01    0.904404
 arc02    0.768640
 arc03    0.910544
 arc04    0.835575
 arc05    0.911111
 arc06    0.943415
 Name: feedback, dtype: float64,
 'equipment_area': equipment_area
 ga00000001    1.000000
 ga00000002    0.928571
 ga00000003    0.924370
 ga00000004    1.000000
 ga00000005    0.903409
                 ...   
 ga00001281    0.851064
 ga00001282    0.983471
 ga00001283    0.872690
 ga00001284    0.768116
 

In [12]:
"""
Optimizes hyper-parameters for NGBoost using Optuna.
"""
import os

from sklearn.model_selection import train_test_split
import optuna
from dotenv import load_dotenv
from ngboost import NGBRegressor
from ngboost.distns import Bernoulli
from ngboost.scores import LogScore
from sklearn.tree import DecisionTreeClassifier
import click
from time import time

load_dotenv()


class NgbOptimizer:

    def __init__(self, study_name: str, X_train, y_train):
        # We use only the train set in optimization. Later we split it to train and validation in each iteration.
        self.X_train, self.y_train = X_train, y_train
        sampler = optuna.samplers.NSGAIISampler()
        pruner = optuna.pruners.SuccessiveHalvingPruner()
        self.study = optuna.create_study(storage=os.environ['OPTUNA_DB_URL'], study_name=study_name,
                                         direction='minimize', load_if_exists=True, sampler=sampler, pruner=pruner)

    def optimize(self):
        self.study.optimize(self._objective, n_trials=10000, catch=(ValueError,), gc_after_trial=True)

    def _objective(self, trial: optuna.Trial):
        print('Splitting train and validation data.')
        start_time = time()
        X_train, X_val, y_train, y_val = train_test_split(self.X_train, self.y_train)
        print(f'Took {time()-start_time:.2f} s.')
        tree_parameters = {
            'criterion': 'friedman_mse',    # No clue if better than MSE or MAE
            'splitter': 'best',
            'max_depth': trial.suggest_int('max_depth', 1, 1000),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 1000),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 1000),
            'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),
            'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 10000),
            'min_impurity_decrease': 0,     # We use other parameters (depth, number of leaves etc.) for regularization
            'ccp_alpha': 0,                 # We use other parameters (depth, number of leaves etc.) for regularization
        }
        print('Initializing base learner.')
        start_time = time()
        tree_learner = DecisionTreeClassifier(**tree_parameters)
        print(f'Took {time()-start_time:.2f} s.')
        ngb_parameters = {
            'Dist': Bernoulli,
            'Score': LogScore,
            'Base': tree_learner,
            'natural_gradient': True,
            'n_estimators': 500,
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1),
            'minibatch_frac': trial.suggest_uniform('minibatch_frac', 1e-5, 1),
            'col_sample': trial.suggest_uniform('col_sample', 1e-5, 1),
            'tol': 0,
            'verbose_eval': 1,
        }
        trial.set_user_attr('tol', 0)
        early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 1, 20)

        # We define val_loss_monitor so that it reports the validation score to the trial and prunes it if necessary.
        # Links to help understand what's happening here:
        # https://optuna.readthedocs.io/en/v1.0.0/tutorial/pruning.html
        # https://github.com/stanfordmlgroup/ngboost/blob/master/ngboost/ngboost.py
        # It's pretty gimmicky, but there doesn't seem to be a better way of doing it.
        step = -1

        def pruning_loss_monitor(manifold, y):
            nonlocal step
            step += 1
            score = manifold.total_score(y)
            trial.report(score, step)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
            return score

        print(f'Initializing NGBRegressor.')
        start_time = time()
        ngb = NGBRegressor(**ngb_parameters)
        print(f'Took {time()-start_time:.2f} s.')
        print(f'Fitting.')
        print(f'NGB parameters: {ngb_parameters}')
        print(f'Tree parameters: {tree_parameters}')
        print(f'early_stopping_rounds: {early_stopping_rounds}')
        start_time = time()
        ngb.fit(X=X_train, Y=y_train, X_val=X_val, Y_val=y_val, early_stopping_rounds=early_stopping_rounds,
                val_loss_monitor=pruning_loss_monitor)
        print(f'Took {time()-start_time:.2f} s.')
        trial.set_user_attr('n_estimators_actualized', len(ngb.base_models))
        log_score = ngb.evals_result['val']['LOGSCORE'][ngb.best_val_loss_itr]
        return log_score


@click.command()
@click.argument('study-name')
def optimize(study_name: str):
    NgbOptimizer(study_name).optimize()


if __name__ == '__main__':
    optimize()


ModuleNotFoundError: No module named 'src'